In [2]:
!pip install pyomo
!pip install cplex -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 15.8 MB/s eta 0:00:00


In [3]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [9]:
from pyomo.core.base.set_types import NonNegativeIntegers
model = pyo.ConcreteModel()

model.lines = pyo.Set(initialize = ['pl1','pl2'])
model.seats = pyo.Set(initialize = ['type1','type2','type3'])

model.S = pyo.Param(model.lines, model.seats, initialize = {('pl1','type1'):20,('pl1','type2'):30,('pl1','type3'):40,
                                                            ('pl2','type1'):50,('pl2','type2'):35,('pl2','type3'):45})

model.P = pyo.Param(model.lines, initialize ={'pl1':1000,'pl2':2000})
model.Sa = pyo.Param(model.lines, initialize ={'pl1':400,'pl2':600})
P = model.P
Sa = model.Sa
S = model.S

model.x = pyo.Var(model.lines, domain = pyo.NonNegativeIntegers)
x = model.x
model.y = pyo.Var(model.lines, domain = pyo.Binary)
y = model.y


# think there's a mistake to have j like this define in function, have better way to code this constraints
def objective_rule(model):
  return sum(P[i]*y[i] for i in model.lines) + sum(Sa[i]*x[i] for i in model.lines)
model.objective = pyo.Objective(rule = objective_rule)

def const1(model,j):
  return sum(S[i,'type1']*x[i] for i in model.lines) >= 120
model.con1 = pyo.Constraint(model.lines, rule = const1)

def const2(model,j):
  return sum(S[i,'type2']*x[i] for i in model.lines) >= 150
model.con2 = pyo.Constraint(model.lines, rule = const2)

def const3(model,j):
  return sum(S[i,'type3']*x[i] for i in model.lines) >= 200
model.con3 = pyo.Constraint(model.lines, rule = const3)

def const4(model,i):
  return x[i] <= 30 * y[i]
model.con4 = pyo.Constraint(model.lines, rule = const4)

solver = SolverFactory('cplex_direct')
results = solver.solve(model)

print(results)
print('Objective function =',model.objective())
for i in model.lines:
  print('Number of workers in line',i,'is equal of',x[i]())




Problem: 
- Name: 
  Lower bound: 3400.0
  Upper bound: 3400.0
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 4
  Number of binary variables: 2
  Number of integer variables: 2
  Number of continuous variables: 0
  Number of nonzeros: None
  Sense: 1
Solver: 
- Name: CPLEX 22.1.1.0
  Status: ok
  Wallclock time: 0.00441741943359375
  Termination condition: optimal
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective function = 3400.0
Number of workers in line pl1 is equal of 6.0
Number of workers in line pl2 is equal of 0.0
